In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc

import datetime
# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math


In [2]:
df = pd.read_csv("../data/viewing_rating.csv", encoding='cp932')

In [3]:
df.start_at=pd.to_datetime(df.start_at)


In [4]:
df["drama_key"] = df.start_at.dt.strftime("%y")+df.start_at.dt.strftime("%m")+"_"+df.TV_station+"_"+df.time_table

In [5]:
df
# df.start_at.dt.strftime("%m")

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,...,Episode10,Episode11,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,...,8.8,9.8,-,-,-,-,9.90%,5-5-0,2019-10-01,1910_CX_月21
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,...,9.7,-,-,-,-,-,9.40%,5-4-0,2019-10-01,1910_CX_火22
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,...,9.6,-,-,-,-,-,7.65%,2003/5/1,2019-10-01,1910_TBS_火22
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,...,13.7,-,-,-,-,-,10.89%,5-4-0,2019-10-01,1910_NTV_水22
4,EX,木22,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,...,19.3,-,-,-,-,-,18.48%,3-6-0,2019-10-01,1910_EX_木22
5,CX,木22,モトカレマニア,5.6,5.2,4.1,3.0,4.6,4.1,4.3,...,-,-,-,-,-,-,4.46%,3-5-0,2019-10-01,1910_CX_木22
6,NTV,木23,CHEAT,4.2,3.0,4.4,3.0,3.1,2.9,3.5,...,3.2,-,-,-,-,-,3.45%,4-5-0,2019-10-01,1910_NTV_木23
7,TBS,金22,4分間のマリーゴールド,10.3,7.8,7.6,6.2,7.1,5.7,7.1,...,9.2,-,-,-,-,-,7.52%,4-5-0,2019-10-01,1910_TBS_金22
8,EX,金23,時効警察はじめました,7.7,6.1,5.9,4.9,6.3,6.2,5.5,...,-,-,-,-,-,-,6.01%,2001/5/1,2019-10-01,1910_EX_金23
9,NTV,土22,俺の話は長い,8.4,9.0,8.9,8.4,8,7.7,7.4,...,10.4,-,-,-,-,-,8.45%,3-6-0,2019-10-01,1910_NTV_土22


In [6]:
# import pandas_profiling as pdp
# pdp.ProfileReport(df)

In [7]:
import json

f = open("../data/drama_info.json", 'r')
print(f)
drama_info_json = json.load(f)

<_io.TextIOWrapper name='../data/drama_info.json' mode='r' encoding='UTF-8'>


In [8]:
# def addDramaInfo (drama_title) 
#     main_actor_only_json[drama_title]
    


In [9]:
print (drama_info_json)

{'1910_CX_月21': {'main_actor1': '', 'main_actor2': ''}, '1910_CX_火22': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_火22': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_水22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_木22': {'main_actor1': '', 'main_actor2': ''}, '1910_CX_木22': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_木23': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_金22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_金23': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_土22': {'main_actor1': '', 'main_actor2': ''}, '1910_EX_土23': {'main_actor1': '', 'main_actor2': ''}, '1910_TBS_日21': {'main_actor1': '', 'main_actor2': ''}, '1910_NTV_日22': {'main_actor1': '', 'main_actor2': ''}, '1907_CX_月21': {'main_actor1': '上野樹里', 'main_actor2': '時任三郎'}, '1907_CX_火21': {'main_actor1': '三浦春馬', 'main_actor2': '芳根京子'}, '1907_TBS_火22': {'main_actor1': '石原さとみ', 'main_actor2': '福士蒼汰'}, '1907_NTV_水22': {'main_actor1': '杏', 'main_actor2': '宮沢氷魚'}, '1907_EX_木21': {'main_act

In [10]:
df["main_actor1"] = ""
df["main_actor2"] = ""

In [11]:
for tmp_index in range(len(df)):
    tmp_drama_key = df.iloc[tmp_index]["drama_key"]
    if tmp_drama_key in drama_info_json.keys():
#         print(drama_info_json[tmp_drama_key]["main_actor1"])
        df["main_actor1"][tmp_index] = drama_info_json[tmp_drama_key]["main_actor1"]
        df["main_actor2"][tmp_index] = drama_info_json[tmp_drama_key]["main_actor2"]
#         print("yes"+tmp_drama_title)
    else:
        print(tmp_drama_key)

In [12]:
df

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,...,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key,main_actor1,main_actor2
0,CX,月21,シャーロック,12.8,9.3,9.9,10.6,9.3,8.3,9.9,...,-,-,-,-,9.90%,5-5-0,2019-10-01,1910_CX_月21,,
1,CX,火22,まだ結婚できない男,11.5,7.7,10.0,9.5,10,8.9,8.5,...,-,-,-,-,9.40%,5-4-0,2019-10-01,1910_CX_火22,,
2,TBS,火22,G線上のあなたと私,7.8,8.8,7.3,6.9,6.7,6.7,8.6,...,-,-,-,-,7.65%,2003/5/1,2019-10-01,1910_TBS_火22,,
3,NTV,水22,同期のサクラ,8.1,9.5,9.3,11.5,11.8,11.7,12.2,...,-,-,-,-,10.89%,5-4-0,2019-10-01,1910_NTV_水22,,
4,EX,木22,ドクターX,20.3,19.0,18.1,17.8,15.9,19.3,18.6,...,-,-,-,-,18.48%,3-6-0,2019-10-01,1910_EX_木22,,
5,CX,木22,モトカレマニア,5.6,5.2,4.1,3.0,4.6,4.1,4.3,...,-,-,-,-,4.46%,3-5-0,2019-10-01,1910_CX_木22,,
6,NTV,木23,CHEAT,4.2,3.0,4.4,3.0,3.1,2.9,3.5,...,-,-,-,-,3.45%,4-5-0,2019-10-01,1910_NTV_木23,,
7,TBS,金22,4分間のマリーゴールド,10.3,7.8,7.6,6.2,7.1,5.7,7.1,...,-,-,-,-,7.52%,4-5-0,2019-10-01,1910_TBS_金22,,
8,EX,金23,時効警察はじめました,7.7,6.1,5.9,4.9,6.3,6.2,5.5,...,-,-,-,-,6.01%,2001/5/1,2019-10-01,1910_EX_金23,,
9,NTV,土22,俺の話は長い,8.4,9.0,8.9,8.4,8,7.7,7.4,...,-,-,-,-,8.45%,3-6-0,2019-10-01,1910_NTV_土22,,


In [13]:
def make_person_key_df (actor_actress):
    actor_actress_list = []
    for column in actor_actress.columns:
        if column == "ranking":
            continue
        actor_actress_list.extend(actor_actress[column].values.tolist())
    print ("元の数"+ str(len(actor_actress_list)))
    # uniqueにする
    actor_actress_list = list(set(actor_actress_list))
    #nan を除去
    actor_actress_list = [actor for actor in actor_actress_list if type(actor) is not float ]
    print ("後の数"+str(len(actor_actress_list)))

    df_pivot = pd.DataFrame( columns=actor_actress_list, index=[str(n) for n in range(2008,2020)])

    #pivot 形式に整形する
    for tmp_name in df_pivot.columns:
        for year in actor_actress.columns:
            if year == "ranking": continue
            if  len(actor_actress[actor_actress[year] == tmp_name].ranking) > 0:
                df_pivot.loc[year][tmp_name] = (actor_actress[actor_actress[year] == tmp_name].ranking).values[0]
    return (df_pivot)
    

In [14]:
df_actress = pd.read_csv("../data/actress_power.csv", encoding='cp932')
df_actress_pivot = make_person_key_df(df_actress)


元の数360
後の数62


In [15]:
df_actor = pd.read_csv("../data/actor_power.csv", encoding='cp932')
df_actor_pivot = make_person_key_df(df_actor)

元の数372
後の数65


In [16]:
df_actress_pivot

,天海祐希,波瑠,松下奈緒,綾瀬はるか,真矢みき,米倉涼子,山口智子,松嶋菜々子,藤原紀香,上戸彩,...,菅野美穂,柴崎コウ,能年玲奈,松たか子,樹木希林,堀北真希,吉瀬美智子,榮倉奈々,永作博美,吉高由里子
2008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010,1,NaN,NaN,6,18,19,11,3,27,4,...,5,NaN,NaN,16,NaN,15,NaN,NaN,NaN,NaN
2011,6,NaN,23,2,20,26,28,8,NaN,3,...,1,NaN,NaN,19,NaN,14,NaN,NaN,29,NaN
2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,9,NaN,29,1,NaN,13,NaN,12,NaN,3,...,6,NaN,25,23,24,2,NaN,NaN,NaN,NaN
2015,8,NaN,NaN,1,NaN,9,NaN,14,NaN,10,...,15,NaN,17,11,NaN,2,27,NaN,NaN,26
2016,7,NaN,NaN,1,NaN,20,NaN,21,NaN,12,...,18,NaN,NaN,17,NaN,6,NaN,NaN,NaN,NaN
2017,8,NaN,NaN,2,NaN,18,NaN,22,NaN,12,...,17,13,NaN,NaN,NaN,15,NaN,26,NaN,21


In [17]:
df_actor_namekey = df_actor_pivot.T.rename_axis('actor_actress').reset_index()

In [18]:
df_actress_namekey = df_actress_pivot.T.rename_axis('actor_actress').reset_index()

In [19]:
df_actress_namekey

,actor_actress,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,天海祐希,NaN,NaN,1,6,NaN,NaN,9,8,7,8,5,9
1,波瑠,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26,25
2,松下奈緒,NaN,NaN,NaN,23,NaN,NaN,29,NaN,NaN,NaN,NaN,NaN
3,綾瀬はるか,NaN,NaN,6,2,NaN,NaN,1,1,1,2,1,1
4,真矢みき,NaN,NaN,18,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,米倉涼子,NaN,NaN,19,26,NaN,NaN,13,9,20,18,15,12
6,山口智子,NaN,NaN,11,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,松嶋菜々子,NaN,NaN,3,8,NaN,NaN,12,14,21,22,18,19
8,藤原紀香,NaN,NaN,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,上戸彩,NaN,NaN,4,3,NaN,NaN,3,10,12,12,11,NaN


In [25]:
df_merged = pd.merge(df, df_actor_namekey,left_on='main_actor1', right_on='actor_actress', how='right')
df_merged = pd.merge(df_merged, df_actor_namekey,left_on='main_actor2', right_on='actor_actress', how='right')
#df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor1', right_on='actor_actress', how='right')
#df_merged = pd.merge(df_merged, df_actress_namekey,left_on='main_actor2', right_on='actor_actress', how='right')

In [26]:
# df_merged[["main_actor1","2017","2016","2014"]]
df_merged

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,CX,火21,TWO WEEKS,8.4,5.9,6.4,5.4,6.2,6.2,6.1,...,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
1,CX,水22,僕のいた時間,11.2,9.4,9.4,8.5,10.4,9.2,9.2,...,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
2,TBS,土20,ブラッディ・マンデイ シーズン2,9.5,7.9,8.3,6.6,8.4,9.1,7.4,...,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
3,NTV,土21,サムライ・ハイスクール,14.0,11.3,9.6,9.6,10.6,11,9.9,...,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
4,TBS,土20,ブラッディ・マンデイ,11.4,11.4,11.3,10.8,12.2,11.6,11.9,...,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
5,NTV,土22,ボイス,12.6,8.4,10.7,9.3,11.9,11.1,10.9,...,15,11,NaN,NaN,23,22,20,25,29,NaN
6,NTV,土21,THE LAST COP,8.6,7.4,8.0,9.8,7.6,9.4,8.2,...,15,11,NaN,NaN,23,22,20,25,29,NaN
7,TBS,日21,ナポレオンの村,12.7,7.4,9.1,8.5,7.9,10.2,6.9,...,15,11,NaN,NaN,23,22,20,25,29,NaN
8,TBS,日21,ルーズヴェルト・ゲーム,14.1,11.8,13.7,13.8,16,14.8,14.5,...,15,11,NaN,NaN,23,22,20,25,29,NaN
9,TBS,金22,TAKE FIVE,12.3,10.5,10.5,11.1,8.6,8.6,9.8,...,15,11,NaN,NaN,23,22,20,25,29,NaN


In [27]:
pd.set_option('display.max_columns', 50)

In [32]:
pd.set_option('display.max_rows', 1000)

In [33]:
df_merged

,TV_station,time_table,drama_title,Episode1,Episode2,Episode3,Episode4,Episode5,Episode6,Episode7,Episode8,Episode9,Episode10,Episode11,Episode12,Episode13,Episode14,Episode15,average,suii,start_at,drama_key,main_actor1,main_actor2,actor_actress,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,CX,火21,TWO WEEKS,8.4,5.9,6.4,5.4,6.2,6.2,6.1,6,6.7,7.7,-,-,-,-,-,6.55%,2004/4/1,2019-07-01,1907_CX_火21,三浦春馬,芳根京子,三浦春馬,NaN,NaN,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
1,CX,水22,僕のいた時間,11.2,9.4,9.4,8.5,10.4,9.2,9.2,11,11.3,11.8,9.1,-,-,-,-,10.07%,2004/4/2,2014-01-01,1401_CX_水22,三浦春馬,多部未華子,三浦春馬,NaN,NaN,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
2,TBS,土20,ブラッディ・マンデイ シーズン2,9.5,7.9,8.3,6.6,8.4,9.1,7.4,7.8,7.7,-,-,-,-,-,-,8.23%,4-4-0,2010-01-01,1001_TBS_土20,三浦春馬,佐藤健,三浦春馬,NaN,NaN,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
3,NTV,土21,サムライ・ハイスクール,14.0,11.3,9.6,9.6,10.6,11,9.9,9.3,10.1,-,-,-,-,-,-,10.70%,2003/4/1,2009-10-01,0910_NTV_土21,三浦春馬,城田優,三浦春馬,NaN,NaN,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
4,TBS,土20,ブラッディ・マンデイ,11.4,11.4,11.3,10.8,12.2,11.6,11.9,10.1,11.1,9.9,13.2,-,-,-,-,11.36%,2004/5/1,2008-10-01,0810_TBS_土20,三浦春馬,吉瀬美智子,三浦春馬,NaN,NaN,30,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN
5,NTV,土22,ボイス,12.6,8.4,10.7,9.3,11.9,11.1,10.9,11.2,10.4,12.9,-,-,-,-,-,10.94%,4-5-0,2019-07-01,1907_NTV_土22,唐沢寿明,真木よう子,唐沢寿明,NaN,NaN,15,11,NaN,NaN,23,22,20,25,29,NaN
6,NTV,土21,THE LAST COP,8.6,7.4,8.0,9.8,7.6,9.4,8.2,7.1,8.4,8.5,-,-,-,-,-,8.31%,5-4-0,2016-10-01,1610_NTV_土21,唐沢寿明,窪田正孝,唐沢寿明,NaN,NaN,15,11,NaN,NaN,23,22,20,25,29,NaN
7,TBS,日21,ナポレオンの村,12.7,7.4,9.1,8.5,7.9,10.2,6.9,-,-,-,-,-,-,-,-,9.08%,2-4-0,2015-07-01,1507_TBS_日21,唐沢寿明,麻生久美子,唐沢寿明,NaN,NaN,15,11,NaN,NaN,23,22,20,25,29,NaN
8,TBS,日21,ルーズヴェルト・ゲーム,14.1,11.8,13.7,13.8,16,14.8,14.5,14.1,17.6,-,-,-,-,-,-,14.51%,4-4-0,2014-04-01,1404_TBS_日21,唐沢寿明,檀れい,唐沢寿明,NaN,NaN,15,11,NaN,NaN,23,22,20,25,29,NaN
9,TBS,金22,TAKE FIVE,12.3,10.5,10.5,11.1,8.6,8.6,9.8,8.7,9.5,11.1,-,-,-,-,-,10.17%,2004/3/2,2013-04-01,1304_TBS_金22,唐沢寿明,松雪泰子,唐沢寿明,NaN,NaN,15,11,NaN,NaN,23,22,20,25,29,NaN
